In [1]:
import os
from datetime import datetime
import pandas as pd
from pathlib import Path

import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from PIL import Image

from models import FruitVeggieClassifier0Acc, FruitVeggieClassifier55Acc

In [2]:
# Hyperparameter
num_classes = 36
num_epochs = 20
batch_size = 32
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Data Augmentation and Normalization

In [3]:
# Define the image transformation with data augmentation
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406, 0.5], std=[0.229, 0.224, 0.225, 0.5])
])

val_test_transform = transforms.Compose([
    transforms.Resize(144),
    transforms.CenterCrop(128),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406, 0.5], std=[0.229, 0.224, 0.225, 0.5])
])

In [4]:
# Define the data loaders
def pil_loader(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGBA')

In [5]:
class ImageFolderWithPaths(ImageFolder):
    """Custom dataset that includes image file paths. Extends torchvision.datasets.ImageFolder"""

    def __getitem__(self, index):
        # This is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # The image file path
        path = self.imgs[index][0]
        # Make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [6]:
data_dir = 'data'
train_data = ImageFolder(root=os.path.join(data_dir, 'train'), transform=train_transform, loader=pil_loader)
val_data = ImageFolder(root=os.path.join(data_dir, 'val'), transform=val_test_transform, loader=pil_loader)
benchmark_data = ImageFolderWithPaths(root=os.path.join(data_dir, 'test'), transform=val_test_transform,
                                      loader=pil_loader)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
benchmark_loader = DataLoader(benchmark_data, batch_size=16, shuffle=False)  #, collate_fn=my_collate)

## Model Architecture

In [7]:
# Option to build the model from scratch or load a saved model
def build_or_load_model(load_model, model):
    path = model.model_path
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    start_epoch = 0

    if load_model and os.path.exists(path):
        checkpoint = torch.load(path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        print("Model loaded successfully.")
    else:
        print("Model built from scratch.")

    return model, optimizer, scheduler, start_epoch

In [8]:
def train_and_validate(model, optimizer, scheduler, start_epoch, num_epochs=num_epochs, patience=5):
    criterion = nn.CrossEntropyLoss()
    best_val_loss = float('inf')
    best_epoch = 0
    epochs_without_improvement = 0
    save_path = model.model_path
    
    for epoch in range(start_epoch, start_epoch + num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()  # zero the parameter gradients

            # forward + backward + optimize
            outputs = model(images) 
            loss = criterion(outputs, labels)
            loss.backward() 
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch + 1}/{start_epoch + num_epochs}], Train Loss: {avg_train_loss:.4f}')

        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        accuracy = 100 * correct / total
        print(f'Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.2f}%')

        scheduler.step(avg_val_loss)
        current_lr = scheduler.optimizer.param_groups[0]['lr']
        print(f'Current learning rate: {current_lr:.6f}')

        # Check for early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_epoch = epoch
            epochs_without_improvement = 0
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'epoch': epoch,
                'loss': running_loss,
            }, save_path)
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(
                    f'Early stopping at epoch {epoch + 1}. Best validation loss: {best_val_loss:.4f} at epoch {best_epoch + 1}.')
                break

In [9]:
load_model = True  # Change this to True if you want to load a pre-trained model
init_model = FruitVeggieClassifier55Acc(num_classes=num_classes).to(device)
net, optimizer, scheduler, start_epoch = build_or_load_model(load_model=load_model, model=init_model)  

Model loaded successfully.


In [10]:
train_and_validate(net, optimizer, scheduler, start_epoch, start_epoch + num_epochs)

Epoch [190/398], Train Loss: 1.2179
Validation Loss: 1.4403, Accuracy: 64.80%
Current learning rate: 0.000001
Epoch [191/398], Train Loss: 1.1730
Validation Loss: 1.4404, Accuracy: 64.80%
Current learning rate: 0.000001
Epoch [192/398], Train Loss: 1.2083
Validation Loss: 1.4405, Accuracy: 64.80%
Current learning rate: 0.000001
Epoch [193/398], Train Loss: 1.2022
Validation Loss: 1.4408, Accuracy: 64.80%
Current learning rate: 0.000000
Epoch [194/398], Train Loss: 1.2284
Validation Loss: 1.4409, Accuracy: 64.80%
Current learning rate: 0.000000
Epoch [195/398], Train Loss: 1.2037
Validation Loss: 1.4412, Accuracy: 64.80%
Current learning rate: 0.000000
Early stopping at epoch 195. Best validation loss: 1.4403 at epoch 190.


## Testing the Model

In [11]:
def evaluate(model, test_loader):
    model.eval()  # Set model to evaluation mode
    data = []
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels, paths in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            # Save the image paths and class labels
            for path, label in zip(paths, predicted):
                class_name = benchmark_data.classes[label]  # Get the class name
                data.append([Path(path).stem, class_name])

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')

    df = pd.DataFrame(data, columns=['id', 'ClassLabel'])
    model_path = model.model_path
    model_name = os.path.splitext(os.path.basename(model_path))[0]
    output_dir = os.path.join("results", os.path.dirname(model_path), model_name)
    os.makedirs(output_dir, exist_ok=True)
    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    output_file = os.path.join(output_dir, f"result_{current_time}.csv")
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"CSV file has been created: {output_file}")

In [12]:
evaluate(net, benchmark_loader)

Accuracy: 57.30%
CSV file has been created: results\models\model_1\result_2024-05-26_18-28-24.csv


## References
- https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
- https://pyimagesearch.com/2021/07/19/pytorch-training-your-first-convolutional-neural-network-cnn/